## HTTP request methods

Come detto, il protocollo HTTP prevede una serie di [messaggi di richiesta](https://it.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Messaggio_di_richiesta) che il client può inviare al server. In particolare i più noti sono

- GET
- POST
- PUT
- DELETE


**PER APPROFINDIRE:** https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods

## Request codes

Il significato primario dei codici di risposta è determinato dalla prima cifra, secondo questo schema:

- `1xx`: *information* - La richiesta è stata ricevuta, proceda chi deve procedere, attenda chi deve attendere.
- `2xx`: *success* - La richiesta è stata ricevuta con successo, compresa ed accettata.
- `3xx`: *redirect* - Il client deve eseguire ulteriori azioni per completare la richiesta.
- `4xx`: *client error* - La richiesta contiene una sintassi errata o non può essere soddisfatta.
- `5xx`: *server error* - Il server non è riuscito a soddisfare una richiesta apparentemente valida.


In generale vale questa regola:

- `1xx`, `2xx`, `3xx` &rarr; NESSUN PROBLEMA
- `4xx`, `5xx` &rarr; C'È QUALCHE PROBLEMA

## Headers

Sia la richiesta sia la risposta oltre che a un "contenuto" vengono accompagnati anche da un cosiddetto _**header**_, ovvero una "intestazione".

In parole povere un [header HTTP](https://en.wikipedia.org/wiki/List_of_HTTP_header_fields) è una sorta di bolla di accompagnamento del messaggio, che contiene dei dati utili ai fini della corretta comunicazione tra client e server sotto forma di coppie *campo* (chiave) e *valore*.

Più precisamente, *campi* dell'header HTTP sono un elenco di parole chiave e valori sotto forma di stringhe, le quali sono inviate e ricevute dal client e dal server ad ogni richiesta e risposta HTTP. Queste intestazioni sono solitamente invisibili all'utente e vengono elaborate e loggate solo dalle rispettive applicazioni del server e del client. Definiscono il modo in cui la connessione e le informazioni inviate/ricevute attraverso essa devono essere interpretate e gestite.